In [4]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 2.9 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 2.6 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f43bc280a4e6c5944a942a669a8e5124dfcd7932a983dd5285005fe556b6c97b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
     |████████████████████████████████| 50 kB 2.6 MB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=172291 sha256=4d2b007267be89a90cd30d461486c1870513524e6e8fa649d31e2f5aee24a20e
  Stored in directory: /root/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Succ

In [32]:
!pip install torchaudio==0.8.1

  Using cached torchaudio-0.8.1-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Using cached torch-1.8.1-cp38-cp38-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.0
    Uninstalling torchaudio-0.12.0:
      Successfully uninstalled torchaudio-0.12.0


In [1]:
from jiwer import wer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [2]:
import torch
import torch.nn as nn
model.lm_head = nn.Sequential(
                nn.Flatten(),
                nn.Linear(249*768,1024),
                nn.ReLU(),
                nn.Linear(1024,1),
                nn.Sigmoid()
)

In [3]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [4]:
import numpy as np
import librosa
from helper_code import *
from tqdm import tqdm
import pywt
from sklearn.preprocessing import MinMaxScaler
import pickle as pk
import glob
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools.utils import  get_class_distribution,get_class_distribution2,get_logger,  murmur_score,compute_weighted_accuracy
sys.path.append("..")
from reader.data_reader_physionet import myDataLoader, myDataset
import gc
#from torch.utils.tensorboard import SummaryWriter
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import feature_extract_raw, get_feature_one, feature_extract_wavelet_raw
import pickle as pk
from sklearn.metrics import f1_score
import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model, save_challenge_model
from tqdm import tqdm

In [5]:
gc.collect()
torch.cuda.empty_cache()
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [6]:
data_folder = '/Data1/physionet/data_split/murmur/train'
dev_folder = '/Data1/physionet/data_split/murmur/test'
verbose = 1

In [7]:
def get_features(data_folder, patient_files_trn, raw = True,rand = False) :
    features = dict()

    features['pw_raw'] = []
    features['raw'] = []
    features['mel1'] = []
    murmurs = np.empty((0,2))#3))
    labels = {}
    
    murmur_classes = ['Present', 'Absent'] # ,'Unknown'
    num_murmur_classes = len(murmur_classes)

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        if get_murmur(current_patient_data) != 'Present' :
            murmur_location_information = 'nan'
        else :
            murmur_location_information = current_patient_data.split('\n')[-15].split()[2].split('+')
        
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)
            filename_location = recording_file.split('_')[1].split('.')[0]
            
            if raw :
                raw1, n_samp = feature_extract_raw(filename,samp_sec=20)
#                raw2, n_samp = feature_extract_wavelet_raw(filename,samp_sec=20)
                features['raw'].extend(raw1)
#                features['pw_raw'].extend(raw2)
            # Extract melspec
            else :
                mel1 = feature_extract_raw_melspec(filename,rand)  # np > shape (n_samp,n_mels,time)(n,64,201)
                
                
                n_samp = len(mel1)
                features['mel1'].extend(mel1)


            # Extract labels and use one-hot encoding.
            
            # 클래스 일단 3개지만 학습 데이터에선 loc 아니면 absent로 봄
            
            current_murmur = np.zeros((1,num_murmur_classes), dtype=int)  
#            if (filename_location in murmur_location_information)  :
#                murmur = get_murmur(current_patient_data)
#            else :
#                murmur = 'Absent'
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[0][j] = 1
            elif murmur == 'Unknown' :
                current_murmur[0][0] = 0
                current_murmur[0][1] = 1
            current_murmur = np.repeat(current_murmur,n_samp,0)
            murmurs = np.concatenate((murmurs,current_murmur),axis=0)
            
    
    labels['murmur'] = murmurs
    labels['outcome'] = []
#    features['pw_raw'] = np.array(features['pw_raw'])
    features['raw'] = np.array(features['raw'])
    
    
    return features, labels

In [8]:
a = [1,0]
b = [0.5,0.5]
print(b.index(0.5))

0


# wav2vec2

In [9]:
# Find the patient data files.
patient_files = find_patient_files(data_folder)
dev_patient_files = find_patient_files(dev_folder)

num_patient_files = len(patient_files)
dev_num_patient_files = len(dev_patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')


murmur_classes = ['Present', 'Absent'] # ,'Unknown'
num_murmur_classes = len(murmur_classes)


# Extract the features and labels.
if verbose >= 1:
    print('Extracting features and labels from the Challenge data...')

features, labels = get_features(data_folder, patient_files)
features_dev, labels_dev = get_features(dev_folder, dev_patient_files)

Extracting features and labels from the Challenge data...


100%|████████████████████████████████████████| 191/191 [00:00<00:00, 422.67it/s]


In [10]:
features['raw'].shape[0]

2545

In [11]:
labels['murmur'].shape

(2545, 2)

In [12]:
#torch.cuda.empty_cache()

In [13]:
#torch.cuda.empty_cache()


log_dir = './log/'
project = 'toy0'
logger = get_logger(log_dir + '/' + project)

#train
label_mur = np.empty(len(labels['murmur']))
for i,l in enumerate(labels['murmur']) :
    try :
        label_mur[i] = int(list(l).index(1))
    except :
        label_mur[i] = 0.5
label_out = np.empty(len(labels['outcome']))
for i,l in enumerate(labels['outcome']) :
    label_out[i] = int(list(l).index(1))
label = {'murmur':label_mur, 'outcome':label_out}

#dev
label_mur_dev = np.empty(len(labels_dev['murmur']))
for i,l in enumerate(labels_dev['murmur']) :
    try :
        label_mur_dev[i] = int(list(l).index(1))
    except :
        label_mur_dev[i] = 0.5
label_out_dev = np.empty(len(labels_dev['outcome']))
for i,l in enumerate(labels_dev['outcome']) :
    label_out_dev[i] = int(list(l).index(1))
label_dev = {'murmur':label_mur_dev, 'outcome':label_out_dev}

#    class_count = [i for i in get_class_distribution(label_mur).values()]
#    class_weights = 1./torch.tensor(class_count, dtype=torch.float)

class_weights = torch.tensor([5/9,4/9],dtype=torch.float)

dataset_train = myDataset(features, label,mode ='murmur')
dataloader_train = myDataLoader(dataset=dataset_train,
                                batch_size=32,
                                shuffle=True,
                                num_workers=0)
all_file_train = len(dataloader_train)
if verbose >= 1:
    print("- {} training samples".format(len(dataset_train)))
    print("- {} training batches".format(len(dataloader_train)))
dataset_dev = myDataset(features_dev, label_dev,mode ='murmur')
dataloader_dev = myDataLoader(dataset=dataset_dev,
                                batch_size=32,
                                shuffle=True,
                                num_workers=0)
all_file_dev = len(dataloader_dev)
if verbose >= 1:
    print("- {} testing samples".format(len(dataset_dev)))
    print("- {} testing batches".format(len(dataloader_dev)))


#nnet = LCNN(mode = 'murmur')
#nnet = nnet.cuda()
nnet = model.cuda()
softmax = nn.Softmax(dim=1)
sig = nn.Sigmoid()

optimizer = optim.Adam(nnet.parameters(), lr=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.97)

#criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
criterion = nn.BCELoss()

# Train the model.
if verbose >= 1:
    print('Training model...')


for iter_ in range(20):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0

    train_epoch_mur_f1 = 0.0

    nnet.train()

    pre_train = []
    label_train = []

    for audio_feature, data_label_torch in tqdm(dataloader_train): 

#            alpha =np.random.randint(4,10)/10
#            lam = np.random.beta(alpha,alpha)
#            audio_feature =  torch.cat([audio_feature,lam*audio_feature + (1.0-lam)*audio_feature2], dim=1)
#            data_label_torch_mur = torch.cat([data_label_torch_mur,lam*data_label_torch_mur + (1.0-lam)*data_label_torch_mur2], dim=0)
#        print(audio_feature.shape)
        torch.autograd.set_detect_anomaly(True)
        audio_feature = audio_feature.permute(1,0)
#        audio_feature = audio_feature.unsqueeze(1)
        audio_feature[audio_feature != audio_feature] = 0
        audio_feature= torch.nan_to_num(audio_feature,nan=1e-6)+1e-6

        audio_feature = audio_feature.float().cuda()

        data_label_torch = data_label_torch.cuda()
        data_label_torch = data_label_torch.long()

#        print(audio_feature.shape)
        outputs = nnet(audio_feature).logits

#        print(type(outputs))
#        print(outputs)
        loss = criterion(outputs.float().squeeze(1), data_label_torch.float().squeeze(1))
#        print(outputs)
#        print(loss)
        
#        y_pred_softmax = softmax(outputs.float())
#        y_pred_softmax = F.log_softmax(outputs.float(), dim = 1)
#        _, output_tags = torch.max(outputs.float(), dim = 1)
#        _, output_tags = torch.max(y_pred_softmax, dim = 1)
#        print(outputs)
        output_tags = torch.ceil(outputs-0.5)
#        print(output_tags)
        output_tags = output_tags.detach().cpu().numpy()#.data.cpu().numpy()
        data_label_torch = data_label_torch.data.cpu().numpy()
        result = np.append(output_tags,data_label_torch,axis=1)
#        print('result : ',result)
        train_mur_f1 = f1_score(data_label_torch, output_tags)#, average='macro')



        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_epoch_mur_f1 += train_mur_f1.item()

        pre_train.extend(list(output_tags))
        label_train.extend(list(data_label_torch))
    pre_train[-1] = 0
    pre_train[-2] = 1
    label_train[-1] = np.array([0])
    label_train[-2] = np.array([1])
#    score_train = compute_weighted_accuracy(np.array(pre_train),np.array(label_train),'murmur')
    score_train =0
    logger.info("Iteration:{0}, loss = {1:.6f}, mur_f1 = {2:.3f}, mur_score = {3:.3f}".format(iter_, running_loss/all_file_train, train_epoch_mur_f1/all_file_train, score_train))

    with torch.no_grad():
        running_loss_dev = 0.0
        dev_epoch_f1 = 0.0


        pre_list = []
        label_list = []
        for audio_feature_dev, data_label_torch_dev  in tqdm(dataloader_dev) : 
            
            audio_feature_dev = audio_feature_dev.permute(1,0)
#            audio_feature_dev = audio_feature_dev.unsqueeze(1)
            audio_feature_dev[audio_feature_dev != audio_feature_dev] = 0
            audio_feature_dev = torch.nan_to_num(audio_feature_dev,nan=1e-6)+1e-6
            audio_feature_dev = audio_feature_dev.float().cuda()           
            label_dev = data_label_torch_dev

            label_dev = label_dev.cuda()
            label_dev = label_dev.long()

            outputs_dev = nnet(audio_feature_dev).logits
#            print(outputs_dev[0])

            loss_dev = criterion(outputs_dev.float().squeeze(1), label_dev.float().squeeze(1))
#            y_pred_softmax = softmax(outputs_dev.float())
#            y_pred_softmax = F.log_softmax(outputs_dev.float(), dim = 1)
#            _, output_tags = torch.max(outputs_dev.float(), dim = 1)
#            _, output_tags = torch.max(y_pred_softmax, dim = 1)
            output_tags = torch.ceil(outputs_dev-0.5)
            output_tags = output_tags.detach().cpu().numpy()#.data.cpu().numpy()
            label_dev = label_dev.data.cpu().numpy()

            dev_f1 = f1_score(label_dev, output_tags)#, average='macro')

            running_loss_dev += loss_dev.item()
            dev_epoch_f1 += dev_f1.item()
            pre_list.extend(list(output_tags))
            label_list.extend(list(label_dev))
        pre_list[-1] = 0
        pre_list[-2] = 1
#        pre_list[-3] = 2
        label_list[-1] = np.array([0])
        label_list[-2] = np.array([1])
#        label_list[-3] = np.array([2])
     
#        score_dev = compute_weighted_accuracy(np.array(pre_list),np.array(label_list),'murmur')
        score_dev = 0
        logger.info("Iteration:{0}, valid_loss = {1:.6f}, valid_mur_f1 = {2:.3f}, valid_mur_score = {3:.3f}".format(iter_, running_loss_dev/all_file_dev, dev_epoch_f1/all_file_dev, score_dev))





- 2545 training samples
- 80 training batches
- 639 testing samples
- 20 testing batches
Training model...


100%|███████████████████████████████████████████| 80/80 [01:25<00:00,  1.06s/it]

08-06 03:16 [MainProcess, 97247] [INFO ]  Iteration:0, loss = 0.433976, mur_f1 = 0.900, mur_score = 0.000



100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.96it/s]

08-06 03:16 [MainProcess, 97247] [INFO ]  Iteration:0, valid_loss = 0.386338, valid_mur_f1 = 0.921, valid_mur_score = 0.000


...

08-06 03:43 [MainProcess, 97247] [INFO ]  Iteration:18, valid_loss = 0.415529, valid_mur_f1 = 0.938, valid_mur_score = 0.000



100%|███████████████████████████████████████████| 80/80 [01:26<00:00,  1.09s/it]

08-06 03:45 [MainProcess, 97247] [INFO ]  Iteration:19, loss = 0.055087, mur_f1 = 0.988, mur_score = 0.000



100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.85it/s]

08-06 03:45 [MainProcess, 97247] [INFO ]  Iteration:19, valid_loss = 0.549353, valid_mur_f1 = 0.939, valid_mur_score = 0.000


In [14]:
model_folder = 'w2v2_model_dnn'
outcome_classes = ['Abnormal','Normal']

In [15]:
save_challenge_model(model_folder, nnet,nnet, murmur_classes, outcome_classes, m_name = 'toy_ti', mel_shape = (100,313) )



In [16]:
def run_challenge_models(model, data, recordings, verbose,thres):
    torch.cuda.empty_cache()
#    if model['model'] == 'toy_ti' :
#        model1 = get_toy(model['mel_shape'])
#    model1.load_weights(model['model_fnm'])
    GPU_NUM = 0 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)


#    if model['model'] == 'toy_ti' :
#        trained_model =  torch.load(model['mur_model_fnm'])
#        trained_model2 =  torch.load(model['out_model_fnm'])
#        print(trained_model)
#        nnet = LCNN(mode = 'murmur')
#        nnet.load_state_dict(trained_model)
#        nnet = nnet.cuda()
#        nnet2 = LCNN(mode = 'murmur')
#        nnet2.load_state_dict(trained_model2)
#        nnet2 = nnet2.cuda()
        #print('*')
    nnet = model
    nnet2 = model
    nnet = nnet.cuda()
    nnet2 = nnet2.cuda()
    nnet.eval()
    nnet2.eval()
#    print(nnet)
        
#    murmur_classes = model['murmur_classes']
#    outcome_classes = model['outcome_classes']
    murmur_classes = ['Present','Unknown', 'Absent'] # 
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    # Load features.
    features = get_feature_one(data, verbose = 0)

    probab1 = np.empty((0,2))
    probab2 = np.empty((0,2))
    for i in range(len(recordings)) :
#        print(recordings[i])
        audio_feature,_ = feature_extract_raw(recordings[i]/ 32768,samp_sec=20)
#        print(type(audio_feature))
        audio_feature = torch.from_numpy(np.array(audio_feature))
        audio_feature[audio_feature != audio_feature] = 0
        audio_feature = torch.nan_to_num(audio_feature,nan=1e-6) +1e-6
        audio_feature = audio_feature.float().cuda()
#        audio_feature = audio_feature.unsqueeze(1)
#        audio_feature = audio_feature.permute(1,0,2)
        
#64, 1, 1, 157
        outputs = nnet(audio_feature).logits
        outputs2 = nnet2(audio_feature).logits
#        print(outputs)
#        print('outputs2 : ',outputs2)
#        softmax = nn.Softmax(dim = 1)
#        outputs_murmur = F.log_softmax(outputs.float(), dim = 1)
#        outputs_outcome = F.log_softmax(outputs2.float(), dim = 1)#
#        outputs_murmur = softmax(outputs)
#        outputs_outcome = softmax(outputs2)
        outputs_murmur = outputs
        outputs_outcome = outputs2
        outputs_murmur = torch.cat((1-outputs_murmur,outputs_murmur),dim = 1)
        outputs_outcome = torch.cat((1-outputs_outcome,outputs_outcome),dim = 1)
#        print('outputs_murmur : ',outputs_murmur)
        prob1 = outputs_murmur.mean(axis = 0).reshape(1,2) 
        prob1 = prob1.data.cpu().numpy()
        probab1 = np.concatenate((probab1,prob1),axis=0)

        prob2 = outputs_outcome.mean(axis = 0).reshape(1,2) 
        prob2 = prob2.data.cpu().numpy()
        probab2 = np.concatenate((probab2,prob2),axis=0)
        
    p1 = probab1.max(axis=0)
    p2 = probab2.max(axis=0)
    
        
    
#    print('p1 :',p1)
#    print('p2 :',p2)
        
        
    # Get classifier probabilities.
#    idx = np.argmax(p1)
    if p1[0]>thres:
        idx = 0
    else :
        idx = 2
#    idx2 = np.argmax(p2)
    if p2[0]>0.25 :
        idx2 = 0
    else :
        idx2 = 1

    n_p1 = np.zeros(len(murmur_classes))
    n_p1[0] = p1[0]
    n_p1[1] = 0
    n_p1[2] = p1[1]
    # Choose label with higher probability.
    labels_murmur = np.zeros(len(murmur_classes), dtype=np.int_)
    labels_murmur[idx] = 1
    labels_outcome = np.zeros(len(outcome_classes), dtype=np.int_)
    labels_outcome[idx2] = 1
    
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((labels_murmur, labels_outcome))
    probabilities = np.concatenate((n_p1, p2))

    return classes, labels, probabilities 

In [17]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for running models for the 2022 Challenge. You can run it as follows:
#
#   python run_model.py model data outputs
#
# where 'model' is a folder containing the your trained model, 'data' is a folder containing the Challenge data, and 'outputs' is a
# folder for saving your model's outputs.

import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model#, run_challenge_model
from tqdm import tqdm

# Run model.
def run_models(model_folder, data_folder, output_folder, allow_failures, verbose,thres):
    torch.cuda.empty_cache()
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')
        
    model = load_challenge_model(model_folder, verbose)
    
    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')
##########################
    # Iterate over the patient files.
    for i in tqdm(range(num_patient_files)):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_models(nnet, patient_data, recordings, verbose,thres) ### Teams: Implement this function!!!

        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')




In [18]:
model_folder = 'w2v2_model_dnn'
output_folder = 'w2v2_output_dnn'
allow_failures = False

In [36]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.5)

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:15<00:00,  1.65s/it]

Done.


In [37]:
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv

In [38]:
!cat scores_w2v2.csv

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.783,0.636,0.483,0.733,0.765,14978.600

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.651,0.664,0.586,0.586,0.583,14216.054

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.957,0.500,0.893
AUPRC,0.883,0.073,0.953
F-measure,0.626,0.000,0.822
Accuracy,0.947,0.000,0.748

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.633,0.668
AUPRC,0.708,0.620
F-measure,0.591,0.582
Accuracy,0.582,0.591


# wav2vec2 threshold

In [19]:
import time
a = list(range(10,78,3))
for t in a :
    run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres = t/100)
    time.sleep(2)
    print('threshold : ',t/100)
    !python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output_dnn scores_w2v2_dnn.csv
    !cat scores_w2v2_dnn.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:20<00:00,  1.68s/it]


Done.
threshold :  0.1
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.509,0.770,0.795,15762.962

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.679,0.000,0.849
Accuracy,0.974,0.000,0.791

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:22<00:00,  1.69s/it]


Done.
threshold :  0.13
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.517,0.780,0.801,16167.544

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.692,0.000,0.858
Accuracy,0.974,0.000,0.806

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


...

Done.
threshold :  0.7
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.559,0.838,0.798,17978.467

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.782,0.000,0.897
Accuracy,0.895,0.000,0.906

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:06<00:00,  1.60s/it]


Done.
threshold :  0.73
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.564,0.843,0.801,18186.507

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.791,0.000,0.901
Accuracy,0.895,0.000,0.914

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:05<00:00,  1.60s/it]


Done.
threshold :  0.76
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.568,0.848,0.803,18185.263

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.800,0.000,0.905
Accuracy,0.895,0.000,0.921

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785


In [20]:
import time
a = list(range(78,100,3))
for t in a :
    run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres = t/100)
    time.sleep(2)
    print('threshold : ',t/100)
    !python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output_dnn scores_w2v2_dnn.csv
    !cat scores_w2v2_dnn.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:06<00:00,  1.61s/it]


Done.
threshold :  0.78
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.570,0.848,0.803,18393.574

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.810,0.000,0.901
Accuracy,0.895,0.000,0.921

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:06<00:00,  1.60s/it]


Done.
threshold :  0.81
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.579,0.859,0.809,18601.128

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.829,0.000,0.909
Accuracy,0.895,0.000,0.935

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


...

Done.
threshold :  0.93
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.582,0.864,0.790,19226.374

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.831,0.000,0.914
Accuracy,0.842,0.000,0.957

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:06<00:00,  1.61s/it]


Done.
threshold :  0.96
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.582,0.864,0.790,19226.374

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.831,0.000,0.914
Accuracy,0.842,0.000,0.957

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:09<00:00,  1.62s/it]


Done.
threshold :  0.99
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.625,0.571,0.859,0.744,20062.190

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.648,0.651,0.591,0.602,0.485,16966.900

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.961,0.500,0.830
AUPRC,0.904,0.073,0.899
F-measure,0.800,0.000,0.913
Accuracy,0.737,0.000,0.978

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.647
AUPRC,0.724,0.579
F-measure,0.525,0.658
Accuracy,0.429,0.785
